In [1]:
import midas.file_reader
from datetime import datetime
import numpy as np
from matplotlib import pyplot as plt
import cygno as cy
import time
import pandas as pd

In [3]:
#definisco la funzione che mi riporta il range attorno al picco
def attorno_al_min(f):
    x=f.index(np.min(f))
    x=int(x)
    start=int(x-10)
    stop=int(x+10)
    return f[start:stop]
    

def graficilentii(full_header,number_of_w_readed): #full_header[0][1]=32, full_header[1][1]=8
    if full_header[0][0]==1:
        singleplot =True
    else:
        singleplot= False
    fig, ax = plt.subplots(full_header[0][1], number_of_w_readed, figsize=(18, 3))
    
    for t in range(0, full_header[0][1]):
        offset = t*full_header[1][1]
        
        for w in range(0, number_of_w_readed):
            if singleplot:
                ax[w].plot(np.linspace(0, full_header[2][1], full_header[2][1]), w_slow[offset], label="t: {:d} w{:d}".format(t,w))
                #plt.ylim(0,2500)
            else:
                ax[t,w].plot(np.linspace(0, full_header[2][1], full_header[2][1]), w_slow[offset], label="t: {:d} w{:d}".format(t,w))
                
            offset+=1
            
    #provo a fare i plot: trovo min e faccio la cumulativa
def istogrammi_lenti(full_header,number_of_w_readed):
    if full_header[0][0]==1:
            singleplot =True
    else:
            singleplot= False
    fig, axs= plt.subplots(full_header[0][1], number_of_w_readed, figsize=(18, 3))

    for t in range(0, full_header[0][1]): #[0][1] scorre gli eventi, full_header[0][1]=32, [1][1]=8
        offset = t*full_header[1][1]
        
        for w in range(0, number_of_w_readed):
             #funzione d'onda pmt
            f=attorno_al_min(w_slow[offset])
            for i in range(len(f)):
                f=np.max(f)-f
                
            cumulative=np.cumsum(f)
            if singleplot:
                axs[w].hist(cumulative,bins=20)
            else:
               axs[t,w].hist(cumulative,bins=20) 
            offset+=1
        
verbose = False
outplot = True # fa i plot
run     = 13360
path    = '/jupyter-workspace/ba-storage/cygno-data/' # disco di bari

mfile = cy.open_mid(run=run, path=path, cloud=False, tag='LNF', verbose=verbose)

# esempio lettura informazioni dall'odb  #######
odb = cy.get_bor_odb(mfile)
corrected  = odb.data['Configurations']['DRS4Correction']
channels_offsets  = odb.data['Configurations']['DigitizerOffset']
header_environment = odb.data['Equipment']['Environment']['Settings']['Names Input']
# esempio di retrive di variabili nell'odb
dslow = pd.DataFrame(columns = header_environment)
########################

# lettura equipment nel file #######
for event in mfile:
    if event.header.is_midas_internal_event():
        print("Saw a special event")
        continue

    bank_names = ", ".join(b.name for b in event.banks.values())
    
    for bank_name, bank in event.banks.items():
            
        if ('DGH0' in bank_name): # PMTs wavform 
            full_header= cy.daq_dgz_full2header(event.banks['DGH0'], verbose=verbose)
            w_fast, w_slow = cy.daq_dgz_full2array(event.banks['DIG0'], full_header, verbose=verbose, 
                                                   corrected=corrected, ch_offset=channels_offsets)
            
            if outplot:
                # digitalizzatore lento commentare se non serve
                graficilentii(full_header,number_of_w_readed=4)
                #istogrammi_lenti(full_header,number_of_w_readed=4)

    #print(event.header.serial_number)
    #print(event.banks.items())   

    if event.header.serial_number == 50: # si ferm dopo i primi 20 eventi
        break

myError: openFileError: /jupyter-workspace/ba-storage/cygno-data/LNF/run13360.mid.gz do not exist